In [181]:
import requests
import time
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import pandas as pd
import re

In [216]:
page = 500
url = "https://www.cupoy.com/MixNewsMongoAction.do?op=getTopMixNewsByBucketGroup&groupid=business_tw&startNewsID=1&len="+str(page)  #財經新聞
resp = requests.get(url)
resp.encoding = 'utf-8'
# print(resp.text)  # 如有需要，列印出網頁原始碼
j = json.loads(resp.text)
items = j['items']
print(len(items))

500


In [223]:
j = json.loads(resp.text)
items = j['items']
titles = []
urls = []
fb_shares_arr = [] 
fb_likes_arr = []
gplus_likes_arr = []
pin_counts_arr = []
linkedin_shares_arr = []
reddit_shares_arr = []
nation_arr = []
for i in tqdm(range(len(items))):
    if items[i]['metrics'].__contains__('socialScorer'):
        titles.append(items[i]['title']) 
    #     urls.append(items[i]['linkurl'])
        nation_arr.append(items[i]['nationID'])
        fb_shares_arr.append(items[i]['metrics']['socialScorer']['fb_shares'])
        fb_likes_arr.append(items[i]['metrics']['socialScorer']['fb_likes'])
        gplus_likes_arr.append(items[i]['metrics']['socialScorer']['gplus_likes'])
        pin_counts_arr.append(items[i]['metrics']['socialScorer']['pin_counts'])
        linkedin_shares_arr.append(items[i]['metrics']['socialScorer']['linkedin_shares'])
        reddit_shares_arr.append(items[i]['metrics']['socialScorer']['reddit_shares'])

print(len(nation_arr))

100%|██████████| 500/500 [00:00<00:00, 36517.30it/s]

498


### Get Publisher from title

In [224]:
publishers = []
for i in tqdm(range(len(titles))):
    x = re.split("(\||\-|\｜)", titles[i])
    length = len(x)
    publish = x[length-1].strip()
    if length == 1: 
        publish = ""
    publishers.append(publish)

100%|██████████| 498/498 [00:00<00:00, 79581.03it/s]


### Remove Publisher from title

In [225]:
non_publisher_titles = []
for i in tqdm(range(len(titles))):
    x = re.split("(\||\-|\｜)", titles[i])
    length = len(x)
#   取0到x-1的字再組起來
    title = ""
    if length == 1:
        title = x[0]
    else:
        j = 0
        #   -2 => 移除 ['｜', '-', '|'] 以及出版者
        for j in range(length-2):
            title = title + x[j]
    non_publisher_titles.append(title)
print(len(non_publisher_titles))

100%|██████████| 498/498 [00:00<00:00, 91492.05it/s]

498


In [227]:
news_dict = {
    "titles": non_publisher_titles,
#     "urls": urls,
    "publishers":publishers,
    "fb_shares":fb_shares_arr,
    "fb_likes": fb_likes_arr,
    "gplus_likes":gplus_likes_arr,
    "pin_counts":pin_counts_arr,
    "linkedin_shares":linkedin_shares_arr,
    "reddit_shares":reddit_shares_arr
}
df = pd.DataFrame(news_dict)

#  Remove Pusblisher is empty
df = df[df['publishers']!=""]
df.sort_values(by=['fb_shares'], inplace=True, ascending=False)
df

,titles,publishers,fb_shares,fb_likes,gplus_likes,pin_counts,linkedin_shares,reddit_shares
415,你今天多賺了幾小時？盤點時間損益表，奪回你最寶貴的時間！,經理人,4824,531,0,0,0,0
231,心理學家的職場觀察：公司裡的 4 種人，最累的是這種,經理人,4633,931,0,0,0,0
255,學經歷漂亮不一定好！能推動公司成長的人才，擁有什麼特質？,經理人,2886,0,0,0,0,0
422,【30 歲迷惘又何妨？】「人生哲學」電影片單，陪你度過灰濛濛的迷惘期,VidaOrange,2876,23,0,0,0,0
423,看完後馬上就能實行！科學證實能帶來幸福感的 12 種行為：學新語言、表達感謝,VidaOrange,2447,26,0,0,0,0
...,...,...,...,...,...,...,...,...
156,談「派對遲到效應」與快速融入陌生團體的技巧,大人學,0,0,0,0,0,0
155,因應武漢肺炎疫情，中小企業紓困貸款最快月底開跑,INSIDE,0,0,0,0,0,0
154,與臺積電搶市佔！傳三星獲高通 5G 新晶片合約,INSIDE,0,0,0,0,0,0
153,疫情預測、學者數據、學術成果、政策地圖，清華大學AMiner團隊上線一系列疫情相關AI大數據產品,機器之心,0,0,0,0,0,0


### 計算TFIDF詞頻
1. 將所有需計算的組成字串
2. jieba進行分詞
3. 使用jieba進行tfidf計算

In [231]:
from tqdm import tqdm
import jieba
import jieba.analyse
jieba.set_dictionary('dict.txt.big')

def caculate_TFIDF(datalist):
    # 將所有需計算的組成字串
    word_string = ""
    for data in datalist:
        word_string = word_string + "，" + data
        
    #  jieba進行分詞
    cut_words = jieba.cut(word_string, cut_all=False)
    words = list(cut_words)
    
   #使用jieba進行tfidf計算
    #  tfidf = jieba.analyse.extract_tags(word_string, 10)
    tfidf = jieba.analyse.extract_tags(word_string, topK=40, withWeight=False, allowPOS=())
    return tfidf



caculate_TFIDF(df['titles'])

Building prefix dict from /Users/fan/Documents/Github/pycrawler-project/dict.txt.big ...
Loading model from cache /var/folders/32/4nbdc94x4w71n8cppyr49lz00000gn/T/jieba.u90c238c6edb188e7c4dcfaeaef1126cb.cache
Loading model cost 2.668 seconds.
Prefix dict has been built successfully.


['疫情',
 '武漢',
 '肺炎',
 '投資',
 '員工',
 '什麼',
 '臺灣',
 '中國',
 '職場',
 '未來',
 '趨勢',
 '工作',
 '觀點',
 '美國',
 '衝擊',
 '2020',
 '財訊',
 '防疫',
 '...',
 '企業',
 '新冠',
 '不會',
 '成為',
 'Google',
 '怎麼',
 '時代',
 '商業',
 '20',
 '老闆',
 '10',
 '影響',
 '成功',
 '學習',
 '媽媽',
 '離職',
 '數位',
 '確診',
 'Anue',
 '主管',
 '掌握']

In [116]:
df.hist(column='publishers')

ValueError: hist method requires numerical columns, nothing to plot.